In [ ]:
import ee
import geemap
ee.Initialize()

Helper functions for grabbing modis imgs

In [ ]:
def zeroPad(n):
    return n.format('%02d')

def getYearMonthDay( y, m, d):

    m = ee.Number(m)
    ms = zeroPad(m)

    d = ee.Number(d)
    ds = zeroPad(d)

    y = ee.Number(y)
    ys = zeroPad(y)
    num = ys.cat(ms).cat(ds)

    return ee.Number.parse(num)

def get_analysis_dates(startJulian, endJulian, analysisPeriod):
    # todo: clean this up now that in class...
    analysisDates = ee.List.sequence(
        startJulian, endJulian, analysisPeriod)
    return analysisDates

def getMODISFire( startDate, endDate):
    #   //Bring in MYD14/MOD14 and combine them
    modisFireAqua = ee.ImageCollection('MODIS/006/MYD14A2').select([0]) \
        .filterDate(startDate, endDate)
    modisFireTerra = ee.ImageCollection('MODIS/006/MOD14A2').select([0]) \
        .filterDate(startDate, endDate)
    modisFire = ee.ImageCollection(modisFireAqua.merge(modisFireTerra))
    def fillEmptyCollections(inCollection, dummyImage):
        dummyCollection = ee.ImageCollection(
            [dummyImage.mask(ee.Image(0))])
        imageCount = inCollection.toList(1).length()
        return ee.ImageCollection(ee.Algorithms.If(imageCount.gt(0), inCollection, dummyCollection))


    #   //Reclassify data and add year, month, day, and a unique year-month-day bands
    def reclassify(img):
        remapped = img.remap([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [
                              0, 0, 0, 0, 0, 0, 0, 1, 2, 3]).byte().rename(['confidence'])

        binary = remapped.gte(1).rename('binary').byte()

        out = remapped.addBands(binary)
        out = out.updateMask(remapped.gte(1))
        return out
    dummyImage = ee.ImageCollection('MODIS/006/MOD14A2').first()
    modisFire = fillEmptyCollections(
        modisFire, dummyImage)
    #   //Recode it, and find the most confident year, month, and day
    modisFire = modisFire.map(reclassify)
    return modisFire



Set up lists of time period range, the analysis period, and days of year range.

In [ ]:
syear = 2000
eyear = 2019
analysisPeriod = 16
startJulian, endJulian = 1, 365

dates = get_analysis_dates(startJulian, endJulian, analysisPeriod)
years = ee.List.sequence(syear,eyear)
# print(years.getInfo())

functions to iterate over dates and get modis fire images

In [ ]:
def per_date(dt, year):
    dt = ee.Number(dt)

    analysisStartJulian = dt
    analysisEndJulian = dt.add(analysisPeriod).subtract(1)
    analysisStartDate = ee.Date.fromYMD(
        year, 1, 1).advance(analysisStartJulian, 'day')
    analysisEndDate = ee.Date.fromYMD(
        year, 1, 1).advance(analysisEndJulian, 'day')
    
    img = getMODISFire(analysisStartDate,analysisEndDate).max()
    startYMD = ee.Number(getYearMonthDay(analysisStartDate.get(
        'year'), analysisStartDate.get('month'), analysisStartDate.get('day')))
    endYMD = ee.Number(getYearMonthDay(analysisEndDate.get(
        'year'), analysisEndDate.get('month'), analysisEndDate.get('day')))
    return img.set({
        'modisStartDate': startYMD,
        'modisEndDate': endYMD,
        'system:time_start': analysisStartDate.millis(),
        }).unmask()

def wrap(year, dates):
    out = dates.map(lambda i: per_date(i,year))
    return out
    
    

In [ ]:
lists_of_images = years.map(lambda i: wrap(i,dates))
col = ee.ImageCollection(lists_of_images.flatten())

In [ ]:
Map = geemap.Map()
Map

add the first image to the map to check output (this case it's all 0's)

In [ ]:

img = col.first()
Map.addLayer(img,{'min':0,'max':1})
# print(col.size().getInfo())
# print(col.first().get('system:time_start').getInfo())

export functions

In [ ]:
def export_modis(image, geometry, name, export_path, scale, crs, test=False,test_export=False):

    task = ee.batch.Export.image.toAsset(
        image=image,
        description=name,
        assetId=f'{export_path}/{name}',
        region=geometry,
        scale=scale,
        crs=crs,
        maxPixels=1e13,
    )
    if test:
        print(name,f'{export_path}/{name}',scale,crs)
        print(image.propertyNames().getInfo())
    elif test_export:
        print('starting test export')
        task.start()
    else:
        print(f'export {name} starting')
        task.start()

def export_image_collection(collection, export_func,
                            geometry=None, export_path=None,
                            exportScale=None, crs=None,
                            test=False, test_export=False):
    if geometry is None:
        geometry = collection.first().geometry()
    collection = collection.sort('system:time_start')
    col_size = collection.size()
    col_list = collection.toList(col_size)
    col_size_local = col_size.getInfo()
    export_descriptions = []
    if test or test_export:
        col_size_local = 1
    for i in range(0, col_size_local):
        img_in = ee.Image(col_list.get(i))
        desc = export_func(img_in, geometry, f"{i}_img",
                           export_path, exportScale, crs, test=test,test_export=test_export)
        export_descriptions.append(desc)

    return export_descriptions

merge ROC and DRC geometries, set export parameters, and initialize exports

set test_export to True to export whole collection. 

In [ ]:

DRC_border = ee.FeatureCollection(
    "projects/ee-karistenneson/assets/BurnedBiomass/DRC_Training/DRC_Border")
ROC_border = ee.FeatureCollection(
    "projects/central-africa-silvacarbon/assets/roc_fire/tables/roc_geom")
region = ROC_border.merge(DRC_border).geometry().bounds()
# Map.addLayer(region)
export_path ='projects/central-africa-silvacarbon/assets/modis_thermal/modis_thermal'
exportScale = 1000
export_image_collection(col,export_modis,region,export_path=export_path,exportScale=exportScale,test_export=True)

Un-needed but kept for record keeping. 

First collection was exported with non-fire values masked. When loaded as a time series in CEO this caused the charts to error for any sample that was masked at any point in time (e.g. time series chart does not handle Null values at this time).

To fix this the previous exported collection was called and then each image was unmasked and re-exported.

In [ ]:
# # unmask and reexport 

# DRC_border = ee.FeatureCollection(
#     "projects/ee-karistenneson/assets/BurnedBiomass/DRC_Training/DRC_Border")
# ROC_border = ee.FeatureCollection(
#     "projects/central-africa-silvacarbon/assets/roc_fire/tables/roc_geom")
# export_path ='projects/central-africa-silvacarbon/assets/modis_thermal/modis_thermal_unmask'
# region = ROC_border.merge(DRC_border).geometry().bounds()
# # Map.addLayer(region)
# in_path ='projects/central-africa-silvacarbon/assets/modis_thermal/modis_thermal'
# col = ee.ImageCollection(in_path).map(lambda i : i.unmask())
# exportScale = 1000
# Map.addLayer(col.first(),{'min':0,'max':1},'unmask')
# # print(col.first().getInfo())
# export_image_collection(col,export_modis,region,export_path=export_path,exportScale=exportScale)